# System of non-linear equations - Coupled chemical equilibria
Author: Björn Dahlgren, Applied Physcial Chemistry, KTH Royal Insitiute of Technology

In this example we will study the equilibria between aqueous cupric ions and ammonia.
We will use [chempy](https://pypi.python.org/pypi/chempy) which is a Python package collecting functions and classes useful for chemistry related problems. We will also make use of SymPy for manipulating and inspecting the formulae encountered.

In [ ]:
from chempy.chemistry import Solute, elements, Equilibrium
from chempy.equilibria import EqSystem, composition_balance, NumSysLin, NumSysLog, NumSysSquare, NumSysLinTanh
import periodictable
from IPython.display import Latex, display
import matplotlib.pyplot as plt
%matplotlib inline
def show(s):  # convenience function
    display(Latex('$'+s+'$'))
import sympy; print(sympy.__version__)  # used for rref

Let's define our species with names and composition, we take help from the Python package [periodictable](https://pypi.python.org/pypi/periodictable):

In [ ]:
substances = [
    Solute(n, latex_name=l, formula=periodictable.formula(n)) for n, l in [
        ('H{+}', 'H^+'), ('HO{-}', 'OH^-'), ('NH3 + H{+}', 'NH_4^+'),
        ('NH3', 'NH_3'), ('H2O', 'H_2O'), ('Cu{2+}', 'Cu^{2+}'), ('Cu{2+}NH3', 'Cu(NH_3)^{2+}'),
        ('Cu{2+}(NH3)2', 'Cu(NH_3)_2^{2+}'), ('Cu{2+}(NH3)3', 'Cu(NH_3)_3^{2+}'),
        ('Cu{2+}(NH3)4', 'Cu(NH_3)_4^{2+}'), ('Cu{2+}(NH3)5', 'Cu(NH_3)_5^{2+}'), 
        ('2Cu{2+} + 2HO{-}', 'Cu_2(OH)_2^{2+}'),
        ('Cu{2+} + 3HO{-}', 'Cu(OH)_3^-'), ('Cu{2+} + 4HO{-}', 'Cu(OH)_4^{2-}'),
        ('Cu{2+} + 2HO{-}', 'Cu(OH_2)(s)'),
    ]] #, CuOHp, CuOH2,
substances[-1].solid = True
substance_names = (Hp, OHm, NH4p, NH3, H2O, Cupp, CuNH31pp, CuNH32pp, CuNH33pp,
         CuNH34pp, CuNH35pp, Cu2OH2pp, CuOH3m, CuOH4mm, CuOH2) = [s.name for s in substances]

Let's see how the Solutes are pretty-printed:

In [ ]:
show(', '.join([s.latex_name for s in substances]))

Let's define some initial concentrations. We will consider different amount of added ammonia in 10 mM solutions of $Cu^{2+}$:

In [ ]:
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 0, NH3: 1.0, Cupp: 1e-2, 
            CuNH31pp: 0, CuNH32pp: 0, CuNH33pp: 0, CuNH34pp: 0, CuNH35pp: 0,
            H2O: 55.5, Cu2OH2pp: 0, CuOH2: 0, CuOH3m: 0, CuOH4mm: 0}

Now, let us define the equilibria, data are from course material at Applied Physcial Chemistry, KTH Royal Insitiute of Technology.

In [ ]:
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
CuOH2_s = Equilibrium({CuOH2: 1}, {Cupp: 1, OHm: 2}, 10**-18.8)
CuOH_B3 = Equilibrium({CuOH2: 1, OHm: 1}, {CuOH3m: 1}, 10**-3.6)
CuOH_B4 = Equilibrium({CuOH2: 1, OHm: 2}, {CuOH4mm: 1}, 10**-2.7)
Cu2OH2 = Equilibrium({Cupp: 2, H2O: 2}, {Cu2OH2pp: 1, Hp: 2}, 10**-10.6 / H2O_c**2)
CuNH3_B1 = Equilibrium({CuNH31pp: 1}, {Cupp: 1, NH3: 1}, 10**-4.3)
CuNH3_B2 = Equilibrium({CuNH32pp: 1}, {Cupp: 1, NH3: 2}, 10**-7.9)
CuNH3_B3 = Equilibrium({CuNH33pp: 1}, {Cupp: 1, NH3: 3}, 10**-10.8)
CuNH3_B4 = Equilibrium({CuNH34pp: 1}, {Cupp: 1, NH3: 4}, 10**-13.0)
CuNH3_B5 = Equilibrium({CuNH35pp: 1}, {Cupp: 1, NH3: 5}, 10**-12.4)
equilibria = w_autop, NH4p_pr, CuNH3_B1, CuNH3_B2, CuNH3_B3, CuNH3_B4, CuNH3_B5, Cu2OH2, CuOH_B3, CuOH_B4, CuOH2_s

Let's see if we can print ``equilibria`` in a human-readable form:

In [ ]:
show(', '.join([s.latex_name for s in substances]))
show('~')
from math import log10
for eq in equilibria:
    ltx = eq.latex(dict(zip(substance_names, substances)))
    show(ltx + '~'*(80-len(ltx)) + 'lgK = {0:12.5g}'.format(log10(eq.param)))  # latex table would be better...

To keep our numerical treatment as simple as possible we will try to avoid representing
$Cu(OH)_2(s)$ explicitly (which is present in the three last equilibria). This is becuase the
system of equations change when precipitation sets in.

However, we do want to keep the second and third from last equilibria, therefore we rewrite
those using the last equiblibrium reaction to represent them only using dissolved species:

In [ ]:
new_eqs = CuOH2_s - CuOH_B3, CuOH2_s - CuOH_B4
[str(_) for _ in new_eqs]

Now it's time to exclude the solid species and replace the last three equilibria with our two new ones:

In [ ]:
#skip_subs, skip_eq = (4, 4) # (0, 0), (1, 1), (3, 3), (4, 4), (11, 9)
skip_subs, skip_eq = (1, 3)
simpl_subs = substances[:-skip_subs]
simpl_eq = equilibria[:-skip_eq] + new_eqs
simpl_c0 = {k.name: init_conc[k.name] for k in simpl_subs}

From the law of mass action we can from the equilbria and from the preservation of mass and charge formulate a non-linear system of equations:

In [ ]:
import sympy as sp
import numpy as np
sp.init_printing()
eqsys = EqSystem(simpl_eq, simpl_subs)
x, i, Ks = sp.symarray('x', eqsys.ns), sp.symarray('i', eqsys.ns), sp.symarray('K', eqsys.nr)
params = np.concatenate((i, Ks))
numsys_lin = NumSysLin(eqsys, ln=sp.log, exp=sp.exp)
numsys_lin.f(x, params)

It turns out that the success of the numerical root finding process for above system of equations is terribly sensitive on the choice of the initial guess. We therefore reformulate the equations in terms of the logarithm of the concentrations:

In [ ]:
numsys_log = NumSysLog(eqsys, ln=sp.log, exp=sp.exp)
f = numsys_log.f(x, params)
f

We can take a peek on the jacobian of this vector:

In [ ]:
sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(x)

The preservation equations of mass and charge actually contain a redundant equation, so currently our system is over-determined:

In [ ]:
len(f), eqsys.ns

We could cast the preservation equations into [reduced row echelon form](https://en.wikipedia.org/wiki/Row_echelon_form) (which would remove one equation), but for now we'll leave this be and rely on the Levenberg-Marquardt algorithm to solve our problem in a least-squares sense. (Levenberg-Marquardt uses [QR-decomposition](https://en.wikipedia.org/wiki/QR_decomposition) internally for which it is acceptable to have overdetermined systems).

Let's solve the equations for our inital concentrations:

In [ ]:
C, sol, sane = eqsys.root(simpl_c0, NumSys=(NumSysLog,))
assert sol['success'] and sane
C

Great, let's now vary the initial concentration of $NH_3$ and plot the equilibrium concentrations of our species:

In [ ]:
import numpy as np
plt.figure(figsize=(20,8))
NH3_varied = np.logspace(-4, 0, 100)
Cout_logC, info, success = eqsys.roots(
    simpl_c0, NH3, NH3_varied, NumSys=(NumSysLog,), plot_kwargs={'latex_names': True})
all(success), sum([nfo['nfev'] for nfo in info]), sum([nfo['njev'] for nfo in info])

But the above diagram is only true if we are below the solubility limit of our neglected $Cu(OH)_2(s)$.

Let's plot the solubility product in the same diagram:

In [ ]:
sol_prod = Cout_logC[:, eqsys.as_substance_index(Cupp)]*Cout_logC[:, eqsys.as_substance_index(OHm)]**2
plt.figure(figsize=(20,6))
plt.loglog(NH3_varied, sol_prod, label='[$%s$][$%s$]$^2$' % (eqsys.substances[Cupp].latex_name,
                                                             eqsys.substances[OHm].latex_name))
plt.loglog(NH3_varied, Cout_logC[:, eqsys.as_substance_index(Hp)], ls=':',
           label='[$%s$]' % eqsys.substances[Hp].latex_name)
plt.loglog([NH3_varied[0], NH3_varied[-1]], [10**-18.8, 10**-18.8], 'k--', label='$K_{sp}(Cu(OH)_2(s))$')
plt.xlabel('[$NH_3$]')
_ = plt.legend()

We see that for a ammonia concentration exceeding ~500-600 mM we would not precipitate $Cu(OH)_2(s)$ even though our pH is quite high (almost 12).

We have solved the above system of equations for the *logarithm* of the concentrations. How big are our absolute and relative errors compared to the linear system? Let's plot them:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,8))
eqsys.plot_errors(Cout_logC, simpl_c0, NH3_varied, NH3, axes=axes)

Not bad. So the problem is essentially solved. But let's say that it is very important to know the exact position of the intersection for the solutbility limit. We can locate it using the secant method:

In [ ]:
from scipy.optimize import newton
convergence = []
def sol_lim(c_NH3):
    c0 = simpl_c0.copy()
    c0[NH3] = c_NH3
    C, sol, sane = eqsys.root(c0, NumSys=(NumSysLog,))
    assert sol['success'] and sane
    prod = C[eqsys.as_substance_index(Cupp)]*C[eqsys.as_substance_index(OHm)]**2
    discrepancy = prod/10**-18.8 - 1
    convergence.append(discrepancy)
    return discrepancy
Climit_NH3 = newton(sol_lim, 0.5)
convergence = np.array(convergence).reshape((len(convergence), 1))
print(convergence)
Climit_NH3

For fun, let's see what the equation system looks like if we canonicalize it by transforming the equations for equibliria and the equations for the preservation relations to their respective reduced row echelon form:

In [ ]:
print(params[14].is_real)

In [ ]:
numsys_log_rref = NumSysLog(eqsys, ln=sp.log, exp=sp.exp, rref_equil=True, rref_preserv=True)
rf = numsys_log_rref.f(x, params)
rf

So the Jacobian should be considerably more diagonally dominant now:

In [ ]:
sp.Matrix(1, len(rf), lambda _, q: rf[q]).jacobian(x)

And let's see if this system converges as well (actually Powell's modified version fails here so we are left with Levenberg-Marquardt)

In [ ]:
plt.figure(figsize=(20,8))
out = eqsys.roots(simpl_c0, NH3, NH3_varied, rref_equil=True,
                  rref_preserv=True, NumSys=(NumSysLog,), plot_kwargs=True)

Sinvce version 0.2.0 of chempy there is support for automatic reformulation of the system of equations when precipitation occurs:

In [ ]:
full_eqsys = EqSystem(equilibria, substances)
def solve_and_plot_full(NumSys, plot_kwargs={'latex_names': True}, **kwargs):
    plt.figure(figsize=(20,8))
    return full_eqsys.roots(init_conc, NH3, NH3_varied, NumSys=NumSys, plot_kwargs=plot_kwargs or {}, **kwargs)
xout_log, sols_log, sane_log = solve_and_plot_full((NumSysLog,), solver='kinsol')

In [ ]:
#out_lin = solve_and_plot_full((NumSysLin,), x0=xout_log[0, :], ) # <-- this currently fails

In [ ]:
out_log_lin = solve_and_plot_full((NumSysLog, NumSysLin)) # <-- this currently fails

In [ ]:
# out_log_sq = solve_and_plot_full((NumSysLog, NumSysSquare))

In [ ]:
plt.subplot(1,1,1)
for idx in range(out_log_lin[0].shape[1]):
    plt.plot(NH3_varied, out_log_lin[0][:, idx])
fig = plt.gcf()

In [ ]:
try:
    from bokeh.plotting import figure, output_notebook, show
    from bokeh.mpl import to_bokeh
    output_notebook()
    show(to_bokeh(fig))
except ImportError:
    print('bokeh not installed')